In [2]:
!mamba install pandas

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 2.1459000000059603 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ pandas                        3.0.0                         np22py313h9d9dc1e_0           emscripten-forge              
+ python-tzdata                 2025.3                        pyhd8ed1ab_0                  conda-forge                   
- pip                           25.3                          pyh145f28c_0                  conda-forge                   


In [3]:
import pandas as pd
import sqlite3

In [4]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
conn = sqlite3.connect(":memory:")

In [7]:
with open("restaurants.sql", "r", encoding="utf-8") as file:
    sql_script = file.read()

conn.executescript(sql_script)

In [8]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [9]:
merged_df = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

In [10]:
final_df = pd.merge(
    merged_df,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [11]:
final_df.shape
final_df.head()
final_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  str    
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  str    
 6   name               10000 non-null  str    
 7   city               10000 non-null  str    
 8   membership         10000 non-null  str    
 9   restaurant_name_y  10000 non-null  str    
 10  cuisine            10000 non-null  str    
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), str(7)
memory usage: 664.1 KB


In [12]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [13]:
final_df.groupby("city")["total_amount"].sum()

city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64

In [14]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [15]:
final_df.groupby("city")["total_amount"].sum()


city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64

In [16]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"].sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [17]:
final_df.groupby("cuisine")["total_amount"].mean()


cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64

In [18]:
final_df.groupby("cuisine")["total_amount"].mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [19]:
final_df.groupby("cuisine")["restaurant_id"].nunique()


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [20]:
(final_df[final_df["membership"] == "Gold"].shape[0] / final_df.shape[0]) * 100


49.87

In [21]:
final_df[final_df["rating"] >= 4.5].shape[0]


3374

In [31]:
final_df.groupby("name") \
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    ) \
    .query("total_orders < 20") \
    .sort_values("avg_order_value", ascending=False)



,avg_order_value,total_orders
name,,
User_2429,1497.42,1
User_889,1492.63,1
User_1843,1484.24,1
User_1882,1481.84,1
User_925,1476.18,1
...,...,...
User_2368,119.08,1
User_2583,117.62,1
User_1188,109.58,1


In [26]:
final_df.groupby(["membership", "cuisine"])["total_amount"].sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [27]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum()


quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [28]:
final_df["user_id"].nunique()


2883

In [29]:
final_df.groupby("user_id")["total_amount"].sum() \
    .loc[lambda x: x > 1000] \
    .count()


np.int64(2544)

In [30]:
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'quarter'],
      dtype='str')